In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import *
import pandas as pd

In [4]:
spark=SparkSession.builder\
        .appName('MySparkApp')\
        .master('local[*]')\
        .getOrCreate()
spark.version

'4.0.1'

In [5]:
customers={
    'customer_id': [1, 2, 3, 4, 5],
    'name': ['Winston', 'Jonathan', 'Annabelle', 'Marwan', 'Khaled']
}

orders = {
    'order_id': [1,2,3,4,5,6,7,8,9,10],
    'order_date': [
        '2020-07-31','2020-07-30','2020-08-29','2020-07-29','2020-06-10',
        '2020-08-01','2020-08-01','2020-08-03','2020-08-07','2020-07-15'
    ],
    'customer_id': [1,2,3,4,1,2,3,1,2,1],
    'product_id': [1,2,3,1,2,1,1,2,3,2]
}

products = {
    'product_id': [1,2,3,4],
    'product_name': ['keyboard','mouse','screen','hard disk'],
    'price': [120,80,600,450]
}


In [7]:
df_customers = spark.createDataFrame(
    [(1, 'Winston'), (2, 'Jonathan'), (3, 'Annabelle'), (4, 'Marwan'), (5, 'Khaled')],
    ['customer_id', 'name']
)

df_orders = spark.createDataFrame([
    (1,'2020-07-31',1,1),
    (2,'2020-07-30',2,2),
    (3,'2020-08-29',3,3),
    (4,'2020-07-29',4,1),
    (5,'2020-06-10',1,2),
    (6,'2020-08-01',2,1),
    (7,'2020-08-01',3,1),
    (8,'2020-08-03',1,2),
    (9,'2020-08-07',2,3),
    (10,'2020-07-15',1,2)
], ['order_id','order_date','customer_id','product_id'])

df_products = spark.createDataFrame(
    [(1,'keyboard',120),(2,'mouse',80),(3,'screen',600),(4,'hard disk',450)],
    ['product_id','product_name','price']
)

In [ ]:
df_orders=df_orders.withColumn('order_date',to_date('order_date'))


In [13]:
df_orders.show()

+--------+----------+-----------+----------+
|order_id|order_date|customer_id|product_id|
+--------+----------+-----------+----------+
|       1|2020-07-31|          1|         1|
|       2|2020-07-30|          2|         2|
|       3|2020-08-29|          3|         3|
|       4|2020-07-29|          4|         1|
|       5|2020-06-10|          1|         2|
|       6|2020-08-01|          2|         1|
|       7|2020-08-01|          3|         1|
|       8|2020-08-03|          1|         2|
|       9|2020-08-07|          2|         3|
|      10|2020-07-15|          1|         2|
+--------+----------+-----------+----------+



In [17]:
w=Window.partitionBy('product_id').orderBy(col('order_date').desc())
df_orders=df_orders.withColumn('first_order_date',first('order_date').over(w))
df_orders=df_orders[df_orders['first_order_date']==df_orders['order_date']]
df_orders.show()

+--------+----------+-----------+----------+----------------+
|order_id|order_date|customer_id|product_id|first_order_date|
+--------+----------+-----------+----------+----------------+
|       6|2020-08-01|          2|         1|      2020-08-01|
|       7|2020-08-01|          3|         1|      2020-08-01|
|       8|2020-08-03|          1|         2|      2020-08-03|
|       3|2020-08-29|          3|         3|      2020-08-29|
+--------+----------+-----------+----------+----------------+



In [19]:
result_df=df_orders.join(df_products,on='product_id',how='left')
result_df=result_df[['product_name','product_id','order_id','order_date']].orderBy('order_date')
result_df.show()

+------------+----------+--------+----------+
|product_name|product_id|order_id|order_date|
+------------+----------+--------+----------+
|    keyboard|         1|       6|2020-08-01|
|    keyboard|         1|       7|2020-08-01|
|       mouse|         2|       8|2020-08-03|
|      screen|         3|       3|2020-08-29|
+------------+----------+--------+----------+



25/10/09 17:29:35 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1038382 ms exceeds timeout 120000 ms
25/10/09 17:29:35 WARN SparkContext: Killing executors is not supported by current scheduler.
25/10/09 17:46:30 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:342)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:132)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$